# Extracting a section from a DOCX file using LLMS and RAG

If your `DOCX` file is properly formatted, you do not need to do every step in this notebook. Current document analyzers such as `unstructured` or `docling` do a great job on that. However, there are cases where the `DOCX` files is not formatted using headings or titles, and in those cases, these tools will not solve the issue of extracting a specific section of the document.

In the case above, extracting the sections of the document with the code below would be a good solution.

```python
from docx import Document

def extract_sections(docx_file, keywords):
    doc = Document(docx_file)
    extracted_sections = {}
    current_heading = None

    for paragraph in doc.paragraphs:
        if paragraph.style.name.startswith('Heading'):
            current_heading = paragraph.text
        elif current_heading and any(keyword in paragraph.text.lower() for keyword in keywords):
            extracted_sections[current_heading] = extracted_sections.get(current_heading, '') + '\n' + paragraph.text

    return extracted_sections

```

However, we are focused on files where there is no section beginning with the formatting `paragraph.style.name.startswith('Heading')`.


# Imports and basic setup

In [27]:
from langchain_community.document_loaders.word_document import (
    UnstructuredWordDocumentLoader,
    Docx2txtLoader,
    )
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import os

# If you want to hide your API keys in an .env file, do something like:
from dotenv import load_dotenv
load_dotenv()

# Example: set your OpenAI API key as an environment variable
#os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# Now retrieve the token from environment variables
hf_token = os.getenv("HF_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set the environment variable that huggingface_hub / transformers will look for
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [28]:
from glob import glob

DOCX_FILES = glob('./data/docx/*')


# Organize the list of files making sure that the same file names are in the same index
DOCX_FILES.sort()


# Loading and Parsing the DOCX
LangChain’s default `UnstructuredFileLoader` will try to parse the file using Unstructured under the hood.

As seen in the another notebook, 

In [29]:
def extract_document_text(file, engine='docx2txt'):
    """
    Extracts the text from a document file.
    """
    if engine == 'docx2txt':
        loader = Docx2txtLoader(file)
        raw_documents = loader.load()
    elif engine == 'unstructured':
        loader = UnstructuredWordDocumentLoader(file)
        raw_documents = loader.load()
    else:
        raise ValueError('Invalid engine')
    return raw_documents

In [30]:
# raw_documents is a list of LangChain Document objects
raw_documents = extract_document_text(DOCX_FILES[0], engine='docx2txt')
print(f"Number of documents: {len(raw_documents)}")
print(raw_documents[0].page_content[:500])  # First 500 characters of the first doc


Number of documents: 1
Latrophilin-2 mediates fluid shear stress mechanotransduction at endothelial junctions







Keiichiro Tanaka1,7,*, Minghao Chen1,7, Andrew Prendergast1, Zhenwu Zhuang1, Ali Nasiri2, Divyesh Joshi1, Jared Hintzen1, Minhwan Chung1, Abhishek Kumar1, Arya Mani1, Anthony Koleske3, Jason Crawford4, Stefania Nicoli1 and Martin A. Schwartz1,5,6,*



1 Yale Cardiovascular Research Center, Section of Cardiovascular Medicine, Department of Internal Medicine, School of Medicine, Yale University, New Haven


In [31]:
# raw_documents is a list of LangChain Document objects
raw_documents = extract_document_text(DOCX_FILES[0], engine='unstructured')
print(f"Number of documents: {len(raw_documents)}")
print(raw_documents[0].page_content[:500])  # First 500 characters of the first doc


Number of documents: 1
Latrophilin-2 mediates fluid shear stress mechanotransduction at endothelial junctions

Keiichiro Tanaka1,7,*, Minghao Chen1,7, Andrew Prendergast1, Zhenwu Zhuang1, Ali Nasiri2, Divyesh Joshi1, Jared Hintzen1, Minhwan Chung1, Abhishek Kumar1, Arya Mani1, Anthony Koleske3, Jason Crawford4, Stefania Nicoli1 and Martin A. Schwartz1,5,6,*

1 Yale Cardiovascular Research Center, Section of Cardiovascular Medicine, Department of Internal Medicine, School of Medicine, Yale University, New Haven, CT 065


We can see a very important difference here, which is the time used by each cell to run. As showed in the `comparing_performance-extraction` notebook, the `docx2text` engine shows an order of magnitude faster extraction than the `unstructured` engine. For this reason, we will keep using `docx2text` in this notebook.

# Splitting into Chunks
We want to break the text into smaller chunks (so that each chunk is not too large for embeddings or retrieval). We can use `RecursiveCharacterTextSplitter`.

Here we are using ```RecursiveCharacterTextSplitter``` to split the text into chunks of 1024 characters. However, using a splitter based on `NLTKTextSplitter` or `SpacyTextSplitter` might be a good option to keep the chunks semantically meaningful.

In [32]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=768,   # characters per chunk
    chunk_overlap=64  # overlap for context
)

text_splitted = text_splitter.split_documents(raw_documents)

print(f"Number of chunks after splitting: {len(text_splitted)}")
print(text_splitted[6].page_content)  # A sample chunk


Number of chunks after splitting: 137
kinases (SFKs) within seconds, resulting in ligand-independent activation of VEGF receptors and downstream events including endothelial cell (EC) alignment in the direction of flow(Collins et al, 2012). However, the mechanisms by which proteins located at cell-cell junctions can sense forces from shear stress exerted on the apical surface is unclear. Previous work suggests that activation of the junctional complex is not primary but rather requires an upstream event, mediated presumably by another mechanosensor (Conway et al, 2013).


# Embedding the section into a vectorstore for RAG

We use here `HuggingFace`'s `sentence-transformers` to embed the text chunks into vectors. We then store these vectors in a `VectorStore` for RAG to use.

One question I always had regarding the embedding tokenizer and the RAG was whether we needed to prepare the embeddings using the same model or tokenizer used for the LLM inference. I asked this and obtained the following answer from ChatGPT:

**Embeddings vs. LLM Inference: They Don’t Have to Match**

In a typical “Retrieval-Augmented Generation” (RAG) flow:

* **Embeddings**: You take text chunks and convert them into embedding vectors (using, e.g., sentence-transformers/all-MiniLM-L6-v2).
* **Vector Store**: You store those embeddings in a vector database (e.g., Chroma).
* **Retrieval**: When a query arrives (e.g., “Give me the Data Availability statement”), you embed that query with the same embedding model and find the most similar chunks.
* **Generation**: You feed those retrieved chunks (as plain text) into your Large Language Model (LLM) to answer the query or provide the final text.
Because the LLM only sees or generates plain text, there’s no requirement that the LLM use the same tokenizer or embedding model you used for retrieval. They are effectively separate steps:

* Embedding Model → Produces vector embeddings
* LLM → Takes text, outputs text

Hence, you can absolutely use sentence-transformers/all-MiniLM-L6-v2 for embedding, and any local or remote LLM for the generation step—OpenAI’s GPT-4, Anthropic’s Claude, or a local Hugging Face model like Falcon, Mistral, LLaMA-2, etc.



In [33]:
# 1) Initialize the embedding model
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2) Create (or load) a Chroma vector store
from langchain.vectorstores import Chroma

persist_directory = "./chroma_db"  # Where to store DB files on disk

# Build the vector store from our docs
vectorstore = Chroma.from_documents(
    documents=text_splitted,
    embedding=embedding_model,
    persist_directory=persist_directory
)


# Define the LLM pipeline

In [34]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)



In [35]:
from langchain.llms.base import LLM
from typing import Optional, List, Any
import re

from langchain.llms.base import LLM
from pydantic import Field, BaseModel
from typing import Optional, List, Any

from langchain.llms.base import LLM
from pydantic import Field
from typing import Optional, List, Any

class CTransformersLLM(LLM):
    """
    A custom LangChain LLM wrapper around a ctransformers-based .gguf model.
    """

    ctransformers_model: Any = Field(default=None)
    temperature: float = Field(default=0.0)
    max_tokens: int = Field(default=256)

    class Config:
        # Let Pydantic store arbitrary Python objects (your ctransformers model).
        arbitrary_types_allowed = True

    def __init__(self, ctransformers_model: Any, **kwargs):
        """ 
        ctransformers_model: The loaded model from ctransformers.AutoModelForCausalLM. 
        """
        super().__init__(**kwargs)
        # Bypass pydantic's immutability checks:
        object.__setattr__(self, "ctransformers_model", ctransformers_model)
        object.__setattr__(self, "temperature", self.temperature)
        object.__setattr__(self, "max_tokens",  self.max_tokens)

    @property
    def _llm_type(self) -> str:
        """A short string identifying this LLM type."""
        return "ctransformers"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        """
        The main generation method that LangChain calls internally.
        `prompt` is the text prompt.
        `stop` is an optional list of stop tokens.
        """
        output = self.ctransformers_model(
            prompt,
            max_new_tokens=self.max_tokens,
            temperature=self.temperature,
        )
        if stop:
            for token in stop:
                if token in output:
                    output = output.split(token)[0]
        return output

    @property
    def _identifying_params(self) -> dict:
        """
        Return a dictionary describing your model. 
        LangChain calls `dict(self._identifying_params)` internally.
        """
        return {
            "model_type": "ctransformers",
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }


In [36]:
from ctransformers import AutoModelForCausalLM

# Replace with any GGUF model path/repo you want
model_id = "TheBloke/Mistral-7B-v0.1-GGUF"
model_file = "mistral-7b-v0.1.Q4_0.gguf"  # Example quant file name
model_type = "mistral"  # ctransformers needs to know the base model type
# model_id = "QuantFactory/Meta-Llama-3-8B-GGUF"
# model_file = "Meta-Llama-3-8B.Q4_0.gguf"
# model_type = "llama"  # ctransformers needs to know the base model type
# This downloads the .gguf file from HF (if not already cached)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    model_file=model_file,
    model_type=model_type,  # ctransformers needs to know the base model type
    # or "llama" for Llama 2, etc.
    # Below are optional arguments:
    # gpu_layers=100% if you want MPS acceleration on macOS with Apple Silicon
    # or `gpu_layers=50` etc. The more layers you put on the GPU, the more VRAM is used.
    # For MPS on macOS, set:
    # force_download=True,  # If you want to force re-download
    gpu_layers=50,
    local_files_only=False # If True, won't download but only use local
)

# prompt = "Explain why 4-bit quantization is beneficial."
# output = llm(prompt, max_new_tokens=200)
# print(output)


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 23172.95it/s]


In [37]:
# Suppose `llm` is your ctransformers model
ctrans_llm = CTransformersLLM(
    ctransformers_model=llm,
    temperature=0.1,
    max_tokens=128
    )

# output = ctrans_llm("Explain the meaning of life in 20 words.")
# print(output)


# Build the retrieval QA chain

In [38]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

In [76]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

qa_chain_local = RetrievalQA.from_chain_type(
    llm=ctrans_llm,         # Our ctransformers-based LLM
    chain_type="refine",    # "stuff", "refine", or "map_reduce"
    retriever=retriever,
)

# Example Queries
query = "Please provide the Data Availability statement from this document. Extract the estatemen exactly as it is written in the document. Make sure not to repeat the estatement twice."
response = qa_chain_local.invoke(query)



In [77]:
print("Data Availability estatement:\n", response["result"])

Data Availability estatement:
 
Data availability

The datasets produced in this study are available in the following databases:

All data: BioStudies S-BIAD928: https://www.ebi.ac.uk/biostudies/studies?query=S-BIAD928

------------
Context information

The datasets produced in this study are available in the following databases:

All data: BioStudies S-BIAD928: https://www.ebi.ac.uk/biostudies/studies?query=S-BIAD92


# We check now how does the OpenAI performance with this method work

In [91]:
from langchain_openai import OpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

openai_llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_retries=2,
    max_tokens=512,
    frequency_penalty=2
)

qa_chain_openai = RetrievalQA.from_chain_type(
    llm=openai_llm,         # Our ctransformers-based LLM
    chain_type="refine",    # "stuff", "refine", or "map_reduce"
    retriever=retriever,
)


In [92]:
# Example Queries
query = "Please provide the Data Availability statement from this document. Extract the estatemen exactly as it is written in the document. Make sure not to repeat the estatement twice."
response = qa_chain_openai.invoke(query)



In [93]:
print(response["result"])



The datasets produced in this study are available in the following databases:

All data: BioStudies S-BIAD928: https://www.ebi.ac.uk/biostudies/studies?query=S-BIAD928


**Ground truth**

```
Data availability
The datasets produced in this study are available in the following databases:
All data: BioStudies S-BIAD928: https://www.ebi.ac.uk/biostudies/studies?query=S-BIAD928
```

In [90]:
# Example Queries
query = """
    At the end of the document you will find the section of the figure legends or figure captions. 
    I want you to extrac literally as it is shown in the text the caption of Figure 1.
    
    Extract the text as it appears in the document.
"""
response = qa_chain_openai.invoke(query)



In [88]:
print(response["result"])



Figure 1. Confocal imaging of intersegmental vessels and dorsal aorta in zebrafish larvae. Larvae were fixed, permeabilized, and stained with primary antibody (1:200 rabbit anti-GFP A-11122, ThermoFisher) overnight at 4°C followed by incubation with secondary antibody (Alexa Fluor® 488 goat anti-rabbit IgG H+L A11034). The images show Gα proteins specific for endothelial flow responses in the vasculature of zebrafish larvae. This figure demonstrates the localization of these proteins within the vasculature using confocal microscopy techniques.



**Ground truth**

```
    Figure 1. Gα proteins specific for endothelial flow responses
    A, Flow-induced endothelial alignment after Gα knockdown.  HUVECs were subjected to fluid shear stress (FSS) at a rate of 12 dynes/cm2 for 16 hours and nuclear orientation quantified as histograms showing the percentage of cells within each 10° of the flow directions from 0° to 90° (see Methods) ****: p<0.0001; one-way ANOVA with Tukey’s multiple comparisons test. B, Src family kinase activation, quantified in C. n=5 for control, Gi knockdown, Gq11 knockdown and n=3 for simultaneous knockdown of Gi and Gq11.  Values are means ± SEM. ****: p<0.0001, ***: p<0.001; one-way ANOVA with Tukey multiple comparison test.  D, Rescue of Gq/11 and Gi knockdown by re-expression of siRNA-resistant versions of the indicated proteins. ****: p<0.0001; one-way ANOVA with Tukey’s multiple comparison test. E, Amino acid sequences of Gαi1, Gαi2, and Gαi3 at the mutation sites of Gαi1 gain-of-function mutant.  F, Rescue of Gq/11 and Gi knockdown with indicated Gα proteins. Each point corresponds to one measurement averaged from >500 cells. N=4. ****: p<0.0001; one-way ANOVA with Tukey’s multiple comparisons test.  G, GINIP pulldown assay for activation of Gαi2 by FSS.  N=3. Results quantified in H.  **: p=0.0185, Student’s t-test.  I, GRK2N pulldown assay for activation of Gq.  N=4, quantified in J. *: p<0.05, Student’s t-test. K, GINIP pulldown assay for FSS-induced activation of wild type and Q306K Gαi1.  N = 4, quantified in L. *: p=0.0304, ***: p=0.0017; Student’s t-test. M, Gi2 activation after PECAM-1 knockdown. HUVECs expressing GluGlu tagged Gi2 were transfected with scrambled siRNA or PECAM-1 siRNA, exposed to FSS, and Gi2 activation assayed as described above, quantified in N. Values are means ± SEM, normalized to input Gα protein levels. ***: p<0.001; Student’s t-test, N=4.
```


Looks like the extraction of the `data availability` section, being short enough and likely not to be repeated for the rest of the paper, was successful. However, the extraction of the `Figure 1` section was not successful. This is likely due to the fact that the section is too long and the embeddings are not able to capture the entire section in a single chunk. Increasing the chunk size does not seem to help either, so we will stop pursuing the figure legend extraction in this way.

## Benchmarking `Data Availability` extraction

Now, having an open model and an Open AI model doing a similar task, we will average their extraction quality over the entire set of documents we have to see how reliable this method of extraction might be on the long term:

SyntaxError: invalid character '°' (U+00B0) (3847400222.py, line 2)